In [18]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('All done')

All done


### Dataframe1

read file

In [102]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df.columns=['PostalCode','Borough','Neighborhood']
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Drop Borough == 'Not assigned' cells

In [103]:
df.drop(df[df['Borough']=='Not assigned'].index,inplace=True)
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


Merge cells with same Postcode and Borough

## Answer to Q1

In [106]:
df1= df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x))
df1=df1.reset_index()
df1

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


For those neighborhood are not assigned, assign them the same value as Borough

In [107]:
index_=df1[df1['Neighborhood']=='Not assigned'].index
df1.loc[index_,'Neighborhood']=df1.loc[index_,'Borough']


In [108]:
df1.shape

(103, 3)

### Dataframe2

In [111]:
geo=pd.read_csv('https://cocl.us/Geospatial_data')
geo.columns=['PostalCode','Latitude','Longitude']
geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Answer to Q2

In [119]:
new=pd.merge(df1,geo,on='PostalCode')
new

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


### Explore and cluster neighorhoods in Toronto

In [116]:
address='Toronto,Canada'
geolocator=Nominatim(user_agent='to_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude,longitude))

The geographical coordinate of Toronto are 43.653963, -79.387207.


Only check for the boroughs that have the word 'Toronto':

In [124]:
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(new['Latitude'],new['Longitude'],new['Borough'],new['Neighborhood']):
    if 'Toronto' in borough:
        label='Neighborhood:{}, Borough:{}'.format(neighborhood,borough)
        label=folium.Popup(label,parse_html=True)
        folium.CircleMarker(
            [lat,lng],
            radius=5,
            popup=label,
            color='blue',
            parse_html=False).add_to(map_toronto)  
map_toronto

In [133]:
CLIENT_ID = 'T3PI1WRTAKDAQWNGUGX534H5C0A2SHAPQNOSRLSN5MQQ2TSD' # your Foursquare ID
CLIENT_SECRET = 'Z2CI3T12IZBUDY5AYRDEISVMWHEZ2W3LVE4VCGLUFWKZWSRG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T3PI1WRTAKDAQWNGUGX534H5C0A2SHAPQNOSRLSN5MQQ2TSD
CLIENT_SECRET:Z2CI3T12IZBUDY5AYRDEISVMWHEZ2W3LVE4VCGLUFWKZWSRG


In [144]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Only consider venues in downtown Toronto

Explore the venues within a radius of 500 meters based on postal code

In [145]:
downtown=new[new['Borough']=='Downtown Toronto']
downtown


,PostalCode,Borough,Neighborhood,Latitude,Longitude
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
51,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
53,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
54,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
58,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
59,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


In [146]:
downtown_venues=getNearbyVenues(names=downtown['PostalCode'],
                                latitudes=downtown['Latitude'],
                                longitudes=downtown['Longitude'])
downtown_venues.head()

,PostalCode,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4W,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,M4W,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,M4W,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,M4W,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,M4X,43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [148]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 211 uniques categories.


In [150]:
downtown_onehot=pd.get_dummies(downtown_venues[['Venue Category']],prefix='',prefix_sep='')
downtown_onehot['PostalCode']=downtown_venues['PostalCode']
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,PostalCode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4W,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,M4X,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by postalcode and taking the mean of the frequency of occurence of each category

In [152]:
downtown_grouped = downtown_onehot.groupby('PostalCode').mean().reset_index()
downtown_grouped

,PostalCode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4W,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,...,0.25,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4X,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,M4Y,0.011628,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.011628,0.000000,...,0.00,0.00,0.000000,0.000000,0.011628,0.000000,0.011628,0.011628,0.000000,0.023256
3,M5A,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.021277,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,M5B,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.010000,0.000000,...,0.00,0.00,0.010000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000
5,M5C,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.010000,0.000000,...,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000
6,M5E,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,...,0.00,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,M5G,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.011628,0.000000,...,0.00,0.00,0.011628,0.000000,0.000000,0.011628,0.000000,0.000000,0.000000,0.011628
8,M5H,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.020000,0.000000,...,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000
9,M5J,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,...,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000


Print each postal code along with the top 5 most common venues

In [166]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
downtown_venues_sorted = pd.DataFrame(columns=columns)
downtown_venues_sorted['PostalCode'] = downtown_grouped['PostalCode']

for ind in np.arange(downtown_grouped.shape[0]):
    downtown_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

downtown_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Park,Playground,Trail,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,M4X,Coffee Shop,Italian Restaurant,Park,Pizza Place,Bakery,Market,Restaurant,Café,Pub,Outdoor Sculpture
2,M4Y,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Gym
3,M5A,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Café,Breakfast Spot,Restaurant,Performing Arts Venue,Chocolate Shop
4,M5B,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Bakery,Burger Joint,Diner,Plaza,Pizza Place


Cluster Postalcodes

In [164]:
# set number of clusters
kclusters = 3

downtown_grouped_clustering = downtown_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [167]:
# add clustering labels
downtown_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(downtown_venues_sorted.set_index('PostalCode'), on='PostalCode')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
51,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,2,Coffee Shop,Italian Restaurant,Park,Pizza Place,Bakery,Market,Restaurant,Café,Pub,Outdoor Sculpture
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Gym
53,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Café,Breakfast Spot,Restaurant,Performing Arts Venue,Chocolate Shop
54,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Bakery,Burger Joint,Diner,Plaza,Pizza Place


## Answer to Q3

In [168]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [170]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[0] + list(range(5, downtown_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,M5V,0,Airport Service,Harbor / Marina,Coffee Shop,Boutique,Boat or Ferry,Rental Car Location,Bar,Plane,Sculpture Garden,Airport


In [171]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[0] + list(range(5, downtown_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,1,Park,Playground,Trail,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [172]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[0] + list(range(5, downtown_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,M4X,2,Coffee Shop,Italian Restaurant,Park,Pizza Place,Bakery,Market,Restaurant,Café,Pub,Outdoor Sculpture
52,M4Y,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Gym
53,M5A,2,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Café,Breakfast Spot,Restaurant,Performing Arts Venue,Chocolate Shop
54,M5B,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Bakery,Burger Joint,Diner,Plaza,Pizza Place
55,M5C,2,Coffee Shop,Café,Restaurant,Hotel,Bakery,Clothing Store,Beer Bar,Cosmetics Shop,Cocktail Bar,Breakfast Spot
56,M5E,2,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Seafood Restaurant,Bakery,Steakhouse,Cheese Shop,Café,Japanese Restaurant
57,M5G,2,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint,Chinese Restaurant,Middle Eastern Restaurant,Juice Bar,Bar
58,M5H,2,Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Restaurant,Burger Joint,Gym
59,M5J,2,Coffee Shop,Aquarium,Café,Italian Restaurant,Hotel,Brewery,Scenic Lookout,Restaurant,Fried Chicken Joint,Music Venue
60,M5K,2,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Deli / Bodega,Bar,American Restaurant,Gastropub,Italian Restaurant


There are 3 clusters of postal code in downtown Toronto. Cluster 2 has most boroughs, and the first common category is Coffee shop or Cafe.